## Discount factor
**Discount factor**  - это параметр, используемый в теории управления и обучении с подкреплением для оценки стоимости будущих вознаграждений. Он обозначается символом $\gamma$ (гамма).

В контексте задачи обучения с подкреплением, такой как у **DQN** алгоритма, фактор дисконтирования применяется к будущим вознаграждениям, чтобы учесть уменьшение их значения с течением времени. Фактор дисконтирования позволяет агенту принимать во внимание не только мгновенные вознаграждения, но и будущие вознаграждения, приводя к более долгосрочной стратегии.

**Формула для дисконтирования будущих вознаграждений** выглядит следующим образом:

 $G_t$ = $R_{t+1}$ + $\gamma$ $R_{t+2}$ + $\gamma$^2 $R_{t+3}$ + $\ldots$ = $\sum_{k=0}^{\infty}$ $\gamma$^k $R_{t+k+1}$

где:
- $G_t$ - дисконтированная сумма вознаграждений (вознаграждение с учетом будущих шагов);
- $R_{t+k+1}$ - вознаграждение, полученное на шаге $t+k+1$;
- $\gamma$ - фактор дисконтирования,  $0 \leq \gamma \leq 1$.

Фактор дисконтирования имеет важное значение при принятии решений в условиях неопределенности, где агенту нужно учитывать как мгновенные, так и будущие вознаграждения, с учетом их временного удаления. Выбор подходящего значения для фактора дисконтирования может влиять на стратегии обучения.


## Long-term и Short-term стратегия

Фактор дисконтирования оказывает сильное влияние на долгосрочность стратегии.
В контексте обучения с подкреплением и стратегий агента, термины "краткосрочные" и "долгосрочные" относятся к временным характеристикам принятия решений агентом.

**Краткосрочные стратегии**:

> Агент, придерживающийся краткосрочной стратегии, ориентируется в основном на текущую информацию и мгновенные вознаграждения.
Такой агент может принимать решения, которые приносят максимальное мгновенное вознаграждение, не уделяя большого внимания будущим шагам. Такой агент может принимать решения, которые приносят максимальное мгновенное вознаграждение, не уделяя большого внимания будущим шагам.




**Долгосрочные стратегии**:


> Агент, придерживающийся долгосрочной стратегии, учитывает будущие вознаграждения и последствия своих действий на протяжении более длительного времени.
Такой агент может предпочесть действия, которые могут не приносить максимальное мгновенное вознаграждение, но могут способствовать достижению более высоких наград в будущем.

Когда говорят о краткосрочных или долгосрочных стратегиях в контексте обучения с подкреплением, обычно имеется в виду, как агент принимает решения, оптимизируя сумму будущих вознаграждений. Фактор дисконтирования γ в уравнении обучения с подкреплением регулирует степень учета будущих вознаграждений, и, таким образом, влияет на то, насколько агент ориентирован на краткосрочные или долгосрочные перспективы.

## Метрики для оценки модели
В данной работе мы рассмотрим некоторые метрики, которые могут быть использованы для оценки обучения модели.
Аналогично первой лабораторной работы мы будем решать задачу в окружении LunarLander-v2, а в качестве модели используем уже знакомый DQN. При этом мы обучим модель несколько разных с разным параметром Discount factor.

Для оценки модели мы рассмотрим следущие метрики:

*   Q-values, их поведение и сходимость.
*   Средняя награда за эпизод.
*   Время затрачиваемое на обучение модели.








### Установка зависимостей

На первом шаге мы начинаем с установки необходимых библиотек. Пакет [gymnasium](https://en.wikipedia.org/wiki/Q-learning#Deep_Q-learning) предоставляет различные среды для обучения с подкреплением, в то время как [stable-baselines3](https://en.wikipedia.org/wiki/Q-learning#Deep_Q-learning) предоставляет реализации различных алгоритмов обучения с подкреплением.

In [ ]:
# Установка необходимых библиотек
!apt install -y swig
!pip install gymnasium[box2d]
!pip install stable-baselines3
!pip install PyVirtualDisplay
!sudo apt-get install xvfb

!pip install 'shimmy>=0.2.1'

Импортируем пакет, которые будут использованы в данной работе


In [ ]:
import torch as th
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy
import time

Q-значения (Q-values) представляют собой оценки или оценочные функции, используемые в контексте обучения с подкреплением для измерения ожидаемой полезности (ценности) принятия определенного действия в конкретном состоянии.

Для получения значений для нашей модели реализуем функцию, которая их извлекает из q-сети.

In [ ]:
def calc_q_values(model: DQN, obs: np.ndarray) -> np.ndarray:
  # Доступ к Q-network
  q_net = model.q_net

  # Конвертируем observation в PyTorch tensor
  device = "cuda" if th.cuda.is_available() else "cpu"
  obs_tensor = th.tensor(obs, dtype=th.float32).to(device)

  # Изменяем размерность
  obs_tensor = obs_tensor.unsqueeze(0)

  #Извлекаем Q-values
  q_values = model.q_net.forward(obs_tensor)

  return q_values.detach().cpu().numpy()[0]

Для наглядности результатов Q-values необходимо создать функцию для составления графика по значениям.

In [ ]:
def plot_q_values(q_values_list):
  # Извлечение отдельных списков для каждого Q-значения
  q1_values, q2_values, q3_values, q4_values = zip(*q_values_list)

  # Построение графика
  plt.figure(figsize=(10, 6))
  plt.plot(q1_values, label='Q1 Values')
  plt.plot(q2_values, label='Q2 Values')
  plt.plot(q3_values, label='Q3 Values')
  plt.plot(q4_values, label='Q4 Values')
  plt.xlabel('Time')
  plt.ylabel('Q-Values')
  plt.title('Convergence of Q-Values over Time')
  plt.legend()
  plt.show()

Следующий блок кода служит для следующих целей:

*   Создание окружения
*   Создание модели
*   Обучение модели
*   Оценка модели до обучения
*   Оценка модели после обучения
*   Измерение времени обучени


In [ ]:
def mean_reward(discount_factor, params = None):
  #Создание окружения
  env = gym.make("LunarLander-v2")


  #Создание модели
  if params:
    model = DQN("MlpPolicy", env, verbose=1, gamma=discount_factor, **params)
  else:
    model = DQN("MlpPolicy", env, verbose=1, gamma=discount_factor)

  #Количество эпизодов для оценки модели
  n_eval_episodes = 250

  #Оценка модели до обучения
  mean_reward, std_reward = evaluate_policy(model, env, deterministic=True, n_eval_episodes=n_eval_episodes)
  print(f"До обучения модели с discount_factor = {discount_factor}, mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

  # Засекаем начальное время
  start_time = time.time()

  #Обучение модели
  model.learn(total_timesteps=100000, log_interval=100)

  # Засекаем время завершения
  end_time = time.time()

  #Оценка модели после обучения
  mean_reward, std_reward = evaluate_policy(model, env, deterministic=True, n_eval_episodes=n_eval_episodes)
  print(f"После обучения модели с discount_factor = {discount_factor}, mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")
  model.save(f"dqn_lunarlander_{discount_factor}")
  del model

  # Вычисляем время обучений
  learn_time = end_time - start_time
  return env, learn_time

Q-values необходимо вычислять для некоторого набора предсказаний модели, реализуем для этого функцию

Рассмотрим 3 значения discount factor

## Модель с стандартными параметрами

In [ ]:
def q_values_calculation(discount_factor, env):
  #Загружаем созданную и созраненную модель
  model = DQN.load(f"dqn_lunarlander_{discount_factor}")

  #В LunarLander окружении возможны только 4 действия
  # 0: do nothing
  # 1: fire left orientation engine
  # 2: fire main engine
  # 3: fire right orientation engine

  action_str = ['nothing', 'left', 'main', 'right']
  q_values_list = []

  obs, info = env.reset()
  for _ in range(500):
      q_val = calc_q_values(model,obs)
      q_values_list.append(q_val)
      action, _states = model.predict(obs, deterministic=True)

      print(f"Q-value состояния nothing={q_val[0]:.2f} left={q_val[1]:.2f} main={q_val[1]:.2f} right={q_val[1]:.2f}")
      print(f"Действие: {action_str[action]}")

      obs, reward, terminated, truncated, info = env.step(action)

  return q_values_list

Изменение значения discount factor (γ) в DQN может оказать существенное воздействие на обучение агента и его стратегию.

Воздействие на краткосрочные или долгосрочные стратегии:

*   При более высоком значении γ (ближе к 1) агент будет больше учитывать будущие вознаграждения, что может способствовать формированию более долгосрочных стратегий.

*  При более низком значении γ (ближе к 0) агент будет более склонен к учету только мгновенных вознаграждений, что может привести к формированию краткосрочных стратегий.


Влияние на стабильность обучения:

*   Высокий γ может сделать обучение более стабильным, особенно в средах с большим количеством шума или случайности.
*   Низкий γ может привести к более нестабильному обучению, так как агент будет менее чувствителен к долгосрочным последствиям своих действий.


Влияние на временные рамки обучения:


*   Различные значения γ могут потребовать различных временных рамок обучения для достижения оптимальных стратегий.
*   Изменение γ может потребовать перенастройки гиперпараметров и количества шагов обучения.


Адаптивность к задаче:
*  Эффективное значение γ может зависеть от конкретной задачи. Например, в задачах с большой неопределенностью и случайностью, более высокий γ может быть полезен.


Экспериментальные результаты:
*  Чтобы определить оптимальное значение γ для вашей конкретной задачи, часто требуется провести эксперименты с различными значениями, а затем анализировать результаты.

In [ ]:
# env = gym.make("LunarLander-v2", render_mode="human")

In [ ]:
# env.spec.max_episode_steps

In [ ]:
discount_factors = [0.01,0.5,0.99]

In [ ]:
environment, time_to_lrn = mean_reward(discount_factors[0])
q_vals = q_values_calculation(discount_factors[0], environment)
plot_q_values(q_vals)
print(f'Время обучения модели при discount_factor = {discount_factors[0]} : {time_to_lrn} секунд.')

#### Закрытие среды после тестирования
По завершении тестирования мы закрываем среду с помощью env.close().

In [ ]:
# Закрытие среды
environment.close()

В рамках самостоятельной работы попробуйте обучить и протестировать модель на другой среды доступной в gymnasium.

In [ ]:
environment, time_to_lrn = mean_reward(discount_factors[1])
q_vals = q_values_calculation(discount_factors[1], environment)
plot_q_values(q_vals)
print(f'Время обучения модели при discount_factor = {discount_factors[1]} : {time_to_lrn} секунд.')

In [ ]:
# Закрытие среды
environment.close()

In [ ]:
environment, time_to_lrn = mean_reward(discount_factors[2])
q_vals = q_values_calculation(discount_factors[2], environment)
plot_q_values(q_vals)
print(f'Время обучения модели при discount_factor = {discount_factors[2]} : {time_to_lrn} секунд.')

In [ ]:
# Закрытие среды
environment.close()

## Модель с оптимальными параметрами

In [ ]:
params = {
    'batch_size': 128,
    'buffer_size': 50000,
    'exploration_final_eps': 0.1,
    'exploration_fraction': 0.12,
    'gradient_steps': -1,
    'learning_rate': 0.00063,
    'learning_starts': 0,
    'policy_kwargs': dict(net_arch=[256, 256]),
    'target_update_interval': 250,
    'train_freq': 4
  }

In [ ]:
environment, time_to_lrn = mean_reward(discount_factors[0], params)
q_vals = q_values_calculation(discount_factors[0], environment)
plot_q_values(q_vals)
print(f'Время обучения модели при discount_factor = {discount_factors[0]} : {time_to_lrn} секунд.')

In [ ]:
# Закрытие среды
environment.close()

In [ ]:
environment, time_to_lrn = mean_reward(discount_factors[1], params)
q_vals = q_values_calculation(discount_factors[1], environment)
plot_q_values(q_vals)
print(f'Время обучения модели при discount_factor = {discount_factors[1]} : {time_to_lrn} секунд.')

In [ ]:
# Закрытие среды
environment.close()

In [ ]:
environment, time_to_lrn = mean_reward(discount_factors[2], params)
q_vals = q_values_calculation(discount_factors[2], environment)
plot_q_values(q_vals)
print(f'Время обучения модели при discount_factor = {discount_factors[2]} : {time_to_lrn} секунд.')

In [ ]:
# Закрытие среды
environment.close()

Мы завершили лабораторную работу, посвященную изучению дисконтного фактора и ряда метрик, связанных с обучением с подкреплением на примере алгоритма DQN.

Для того чтобы закрепить полученные знания, предлагается подобрать наиболее оптимальный параметр для рассмторенной задачи или самостоятельно реализовать решение другой задачи, настраивая дисконтный фактор и анализируя изученные метрики. Это позволит лучше понять влияние дисконтного фактора на обучение агента в различных контекстах и освоить навыки тюнинга параметров для достижения оптимальных результатов.
